In [4]:
import tweepy
import pandas as pd
from os import mkdir, listdir
from contextlib import suppress
from utils import get_twitter_api
from tweepy.errors import NotFound, Forbidden
import json

MAX_TWEETS = 2000
MAX_REQUESTS = 100

api = get_twitter_api()

In [2]:
def save_tweets(news_id, tweet_ids, path):
    n = len(tweet_ids)
    idx_list = list(range(0, n, MAX_REQUESTS)) + [n]

    for idx, i in enumerate(idx_list[:-1]):
        chunk = tweet_ids[i:idx_list[idx+1]]

        status_list = api.lookup_statuses(id=chunk)
        retweets_list = []
        
        # get retweets
        for tweet in status_list:
            if tweet.retweet_count:
                with suppress(NotFound, Forbidden):
                    retweets = api.get_retweets(tweet.id)
                    # add field to retrieve ID of original tweet
                    for x in retweets:
                        x._json['retweed_from'] = tweet.id
                    
                    retweets_list.extend(retweets)

        if len(retweets_list):
            print(f"Found a total of {len(retweets_list)} retweets.")

            json_path = f"{path}/retweets/{news_id}"
            mkdir(json_path)

            for retweet in retweets_list:
                with open(f"{json_path}/{retweet.id_str}.json", "w") as f:
                    json.dump(retweet._json, f)
        
        continue

        json_path = f"{path}/tweets/{news_id}"
        mkdir(json_path)

        for tweet in status_list:
            with open(f"{json_path}/{tweet.id_str}.json", "w") as f:
                json.dump(tweet._json, f)

In [5]:
for dataset in ["politifact"]:
    for news_type in ["real", "fake"]:
        df = pd.read_csv(f"dataset/FakeNewsNet/{dataset}_{news_type}.csv")
        df = df[~df['tweet_ids'].isnull()]
        n = len(df)
        
        for i, (news_id, url, title, tweet_ids) in df.iterrows():
            if i % 100 == 0:
                print(f"Processed {i} of {n}")
            
            # convert tweet ids string to ints
            tweet_ids = list(map(int, tweet_ids.split('\t')))[:MAX_TWEETS]

            with suppress(FileExistsError):
                path = f"./dataset/FakeNewsNet/{dataset}/{news_type}"
                print(f"{news_id}: {len(tweet_ids)} tweets")
                save_tweets(news_id, tweet_ids, path)


Processed 0 of 409
politifact14984: 1174 tweets


Rate limit reached. Sleeping for: 192


Found a total of 100 retweets.
Found a total of 78 retweets.
politifact12944: 51 tweets
Found a total of 26 retweets.
politifact779: 8 tweets
Found a total of 0 retweets.
politifact14064: 30 tweets
Found a total of 26 retweets.
politifact14474: 96 tweets
Found a total of 57 retweets.
politifact1313: 1882 tweets


Rate limit reached. Sleeping for: 871
